# IFC OpenShell Voice Assistant - Prototype #5


In [ ]:
%%html
<iframe src="https://ifcjs.github.io/web-ifc-viewer/example/index" width="500" height="300"></iframe>

In [ ]:
import ifcopenshell
import numpy
import apiai
import speech_recognition as sr
import re
import pyttsx3
import webbrowser
import uuid
import time
import tempfile


sensitive_keywords = ["colour", "dimension", "element", "file"]
colours = ["red","blue","yellow"]
dimensions = ["x","y","height"]
elements = ["wall","window","door"]
files = ["4","5","6"]

def text_to_voice (text):

    engine = pyttsx3.init();
    engine.say(text);
    # engine.setProperty('voice', voices[2].id)
    engine.setProperty("rate", 155)
    engine.runAndWait() ;


def get_voice ():

    raw_text = {}
    r = sr.Recognizer()
    mic = sr.Microphone()
    with mic as source:
        r.adjust_for_ambient_noise(source, duration=1)
        r.pause_threshold = 0.8
        r.dynamic_energy_threshold = True
        audio = r.listen(source)
        raw_text = r.recognize_google(audio)
    return raw_text


def nlp(raw_text, sensitive_keywords):

    command_type = [x for x in sensitive_keywords if x in raw_text]
    if command_type[0] == 'colour':
        command_sup = [x for x in colours if x in raw_text]
    elif command_type[0] == 'dimension':
        command_sup = [x for x in dimensions if x in raw_text]
    elif command_type[0] == 'element':
        command_sup = [x for x in elements if x in raw_text]
    elif command_type[0] == 'file':
        command_sup = [x for x in files if x in raw_text]
    return command_type, command_sup



def open_file():

    text_to_voice (f' Can you please give me the file name')
    get_voice_text = get_voice()
    file_name_ = re.findall('\d*\.?\d+',get_voice_text)[0]

    if [x for x in file_name_ if x in files]:
        
        text_to_voice (f'Ok, I will open the file {file_name_}')
        webbrowser.open(f'{file_name_}.txt')

    else:
        
        while bool([x for x in file_name_ if x in files])==False:
            
            text_to_voice (f' Sorry, but I can only open file {files}, Can you please give me the file name?')
            get_voice_text = get_voice()
            file_name_ = re.findall('\d*\.?\d+',get_voice_text)[0]
            if [x for x in file_name_ if x in files]:
                text_to_voice (f'Ok, I will open the file {file_name_}')
                webbrowser.open(f'{file_name_}.txt')

def change_wall_color():
    
    text_to_voice (f' Can you please give me the desired colour')
    get_voice_text = get_voice()
    colour_name_ = [x for x in get_voice_text if x in colours]

    if [x for x in colours if x in get_voice_text]:
        
        text_to_voice (f'Ok, I will change the colour to {colour_name_}')
        """a command to change the colour of the wall"""
        
    else:
        
        while bool([x for x in colours if x in get_voice_text])==False:
            
            text_to_voice (f' Sorry, but I can only change to colour to {colours},Can you please give me the desired colour?')
            get_voice_text = get_voice()
            colour_name_ = [x for x in colours if x in get_voice_text]
            
            if [x for x in colour_name_ if x in colours]:
                
                text_to_voice (f'Ok, I will change the colour to {colour_name_}')
                """a command to change the colour of the wall"""




""" This block of code creates a wall"""

#Variables directions
O = 0., 0., 0.
X = 1., 0., 0.
Y = 0., 1., 0.
Z = 0., 0., 1.


#open a file/Import this from mail file somehow!!!!
ifcfile = ifcopenshell.open("./one.ifc")

# Obtain references to instances defined in template !!! Define the filename with code
owner_history = ifcfile.by_type("IfcOwnerHistory")[0]
project = ifcfile.by_type("IfcProject")[0]
context = ifcfile.by_type("IfcGeometricRepresentationContext")[0]
filename = "one.ifc"


# Helper function definitions

# Creates an IfcAxis2Placement3D from Location, Axis and RefDirection specified as Python tuples
def create_ifcaxis2placement(ifcfile, point=O, dir1=Z, dir2=X):
    point = ifcfile.createIfcCartesianPoint(point)
    dir1 = ifcfile.createIfcDirection(dir1)
    dir2 = ifcfile.createIfcDirection(dir2)
    axis2placement = ifcfile.createIfcAxis2Placement3D(point, dir1, dir2)
    return axis2placement

# Creates an IfcLocalPlacement from Location, Axis and RefDirection, specified as Python tuples, and relative placement
def create_ifclocalplacement(ifcfile, point=O, dir1=Z, dir2=X, relative_to=None):
    axis2placement = create_ifcaxis2placement(ifcfile,point,dir1,dir2)
    ifclocalplacement2 = ifcfile.createIfcLocalPlacement(relative_to,axis2placement)
    return ifclocalplacement2

# Creates an IfcPolyLine from a list of points, specified as Python tuples
def create_ifcpolyline(ifcfile, point_list):
    ifcpts = []
    for point in point_list:
        point = ifcfile.createIfcCartesianPoint(point)
        ifcpts.append(point)
    polyline = ifcfile.createIfcPolyLine(ifcpts)
    return polyline
    
# Creates an IfcExtrudedAreaSolid from a list of points, specified as Python tuples
def create_ifcextrudedareasolid(ifcfile, point_list, ifcaxis2placement, extrude_dir, extrusion):
    polyline = create_ifcpolyline(ifcfile, point_list)
    ifcclosedprofile = ifcfile.createIfcArbitraryClosedProfileDef("AREA", None, polyline)
    ifcdir = ifcfile.createIfcDirection(extrude_dir)
    ifcextrudedareasolid = ifcfile.createIfcExtrudedAreaSolid(ifcclosedprofile, ifcaxis2placement, ifcdir, extrusion)
    return ifcextrudedareasolid
    
create_guid = lambda: ifcopenshell.guid.compress(uuid.uuid1().hex)

# IFC hierarchy creation
site_placement = create_ifclocalplacement(ifcfile)
site = ifcfile.createIfcSite(create_guid(), owner_history, "Site", None, None, site_placement, None, None, "ELEMENT", None, None, None, None, None)

building_placement = create_ifclocalplacement(ifcfile, relative_to=site_placement)
building = ifcfile.createIfcBuilding(create_guid(), owner_history, 'Building', None, None, building_placement, None, None, "ELEMENT", None, None, None)

storey_placement = create_ifclocalplacement(ifcfile, relative_to=building_placement)
elevation = 0.0
building_storey = ifcfile.createIfcBuildingStorey(create_guid(), owner_history, 'Storey', None, None, storey_placement, None, None, "ELEMENT", elevation)

container_storey = ifcfile.createIfcRelAggregates(create_guid(), owner_history, "Building Container", None, building, [building_storey])
container_site = ifcfile.createIfcRelAggregates(create_guid(), owner_history, "Site Container", None, site, [building])
container_project = ifcfile.createIfcRelAggregates(create_guid(), owner_history, "Project Container", None, project, [site])

#Wall Input Conversations
def create_wall():

    text = []
    text_to_voice (f"Please enter X dimension\n")
    text = get_voice()
    x_dimension = float(re.findall('\d*\.?\d+', text)[0])
    # text_to_voice (f" X dimension is {x_dimension}")
    text_to_voice (f" OK")

    text = []
    text_to_voice (f"Please enter y dimension\n")
    text = get_voice()
    y_dimension = float(re.findall('\d*\.?\d+', text)[0])
    #print(y_dimension)
    # text_to_voice (f" Y dimension is {y_dimension}")
    text_to_voice (f" OK")

    text = []
    text_to_voice (f"Please enter the height\n")
    text = get_voice()
    z_dimension = float(re.findall('\d*\.?\d+', text)[0])
    #print(z_dimension)
    # text_to_voice (f" Height is {z_dimension}")
    text_to_voice (f" OK")

    text = []
    text_to_voice (f"Please give me the x and y values of the closest point to origin\n")
    text = get_voice()
    origin_x = float(re.findall('\d*\.?\d+', text)[0])
    origin_y = float(re.findall('\d*\.?\d+', text)[1])
    # text_to_voice (f"Closest point is {origin_x} comma {origin_x}")
    text_to_voice (f" OK, I will create the element")
    # text_to_voice (f" OK, now I can add a wall that its X dimension is {x_dimension}, y dimension is {y_dimension}, height is {z_dimension}, and the origin coordination is {origin_x} ans {origin_x} ")

    #coordinate creation
    point_list_extrusion_area = [(origin_x, origin_y, 0.0), (x_dimension + origin_x, origin_y, 0.0), (x_dimension + origin_x, y_dimension + origin_y, 0.0), (origin_x, y_dimension+ origin_y, 0.0), (origin_x, origin_y, 0.0)]

    #Create Wall
    wall_placement = create_ifclocalplacement(ifcfile, relative_to=storey_placement)
    polyline = create_ifcpolyline(ifcfile, [(0.0, 0.0, 0.0), (5.0, 0.0, 0.0)])
    axis_representation = ifcfile.createIfcShapeRepresentation(context, "Axis", "Curve2D", [polyline])

    extrusion_placement = create_ifcaxis2placement(ifcfile, (0.0, 0.0, 0.0), (0.0, 0.0, 1.0), (1.0, 0.0, 0.0))
    solid = create_ifcextrudedareasolid(ifcfile, point_list_extrusion_area, extrusion_placement, (0.0, 0.0, 1.0), z_dimension)
    body_representation = ifcfile.createIfcShapeRepresentation(context, "Body", "SweptSolid", [solid])

    product_shape = ifcfile.createIfcProductDefinitionShape(None, None, [axis_representation, body_representation])

    wall = ifcfile.createIfcWallStandardCase(create_guid(), owner_history, "Wall", "An awesome wall", None, wall_placement, product_shape, None)

    # Write the contents of the file to disk
    ifcfile.write(filename)

    return x_dimension, y_dimension, z_dimension, origin_x, origin_y

             
def change_dimension():
    pass


def create_element():
        # create_wall()
    if command_sup[0] == 'wall':
        create_wall()
        
    elif command_sup[0] == 'window':
        text_to_voice ('Sorry But I am only one semester old, and cannot do that now!')
        
    elif command_sup[0] == 'door':
        text_to_voice ('Sorry But I am only one semester old, and cannot do that now!')


def get_action(command_type):

    if command_type[0] == 'colour':
        change_wall_color()
        
    elif command_type[0] == 'dimension':
        change_dimension()
        
    elif command_type[0] == 'element':
        create_element()
        
    elif command_type[0] == 'file':
        open_file()     


def voice_to_action():
    text_to_voice ('Hi, how can I help you')
    raw_text = get_voice()
    command_type, command_sup = nlp(raw_text, sensitive_keywords)
    get_action(command_type)

In [ ]:
text_to_voice ('Hi')

In [ ]:
raw_text = get_voice()


In [ ]:
raw_text

In [ ]:
command_type, command_sup = nlp(raw_text, sensitive_keywords)

print(command_type, command_sup)

In [ ]:
def voice_to_action():
    text_to_voice ('Hi, how can I help you')
    raw_text = get_voice()
    command_type, command_sup = nlp(raw_text, sensitive_keywords)
    get_action(command_type)

In [ ]:
import uuid
import time
import tempfile

import ifcopenshell
import numpy
import apiai
import speech_recognition as sr
import re


#Variables directions
O = 0., 0., 0.
X = 1., 0., 0.
Y = 0., 1., 0.
Z = 0., 0., 1.


#open a file/Import this from mail file somehow!!!!
ifcfile = ifcopenshell.open("./one.ifc")

# Obtain references to instances defined in template !!! Define the filename with code
owner_history = ifcfile.by_type("IfcOwnerHistory")[0]
project = ifcfile.by_type("IfcProject")[0]
context = ifcfile.by_type("IfcGeometricRepresentationContext")[0]
filename = "one.ifc"


# Helper function definitions

# Creates an IfcAxis2Placement3D from Location, Axis and RefDirection specified as Python tuples
def create_ifcaxis2placement(ifcfile, point=O, dir1=Z, dir2=X):
    point = ifcfile.createIfcCartesianPoint(point)
    dir1 = ifcfile.createIfcDirection(dir1)
    dir2 = ifcfile.createIfcDirection(dir2)
    axis2placement = ifcfile.createIfcAxis2Placement3D(point, dir1, dir2)
    return axis2placement

# Creates an IfcLocalPlacement from Location, Axis and RefDirection, specified as Python tuples, and relative placement
def create_ifclocalplacement(ifcfile, point=O, dir1=Z, dir2=X, relative_to=None):
    axis2placement = create_ifcaxis2placement(ifcfile,point,dir1,dir2)
    ifclocalplacement2 = ifcfile.createIfcLocalPlacement(relative_to,axis2placement)
    return ifclocalplacement2

# Creates an IfcPolyLine from a list of points, specified as Python tuples
def create_ifcpolyline(ifcfile, point_list):
    ifcpts = []
    for point in point_list:
        point = ifcfile.createIfcCartesianPoint(point)
        ifcpts.append(point)
    polyline = ifcfile.createIfcPolyLine(ifcpts)
    return polyline
    
# Creates an IfcExtrudedAreaSolid from a list of points, specified as Python tuples
def create_ifcextrudedareasolid(ifcfile, point_list, ifcaxis2placement, extrude_dir, extrusion):
    polyline = create_ifcpolyline(ifcfile, point_list)
    ifcclosedprofile = ifcfile.createIfcArbitraryClosedProfileDef("AREA", None, polyline)
    ifcdir = ifcfile.createIfcDirection(extrude_dir)
    ifcextrudedareasolid = ifcfile.createIfcExtrudedAreaSolid(ifcclosedprofile, ifcaxis2placement, ifcdir, extrusion)
    return ifcextrudedareasolid
    
create_guid = lambda: ifcopenshell.guid.compress(uuid.uuid1().hex)

# IFC hierarchy creation
site_placement = create_ifclocalplacement(ifcfile)
site = ifcfile.createIfcSite(create_guid(), owner_history, "Site", None, None, site_placement, None, None, "ELEMENT", None, None, None, None, None)

building_placement = create_ifclocalplacement(ifcfile, relative_to=site_placement)
building = ifcfile.createIfcBuilding(create_guid(), owner_history, 'Building', None, None, building_placement, None, None, "ELEMENT", None, None, None)

storey_placement = create_ifclocalplacement(ifcfile, relative_to=building_placement)
elevation = 0.0
building_storey = ifcfile.createIfcBuildingStorey(create_guid(), owner_history, 'Storey', None, None, storey_placement, None, None, "ELEMENT", elevation)

container_storey = ifcfile.createIfcRelAggregates(create_guid(), owner_history, "Building Container", None, building, [building_storey])
container_site = ifcfile.createIfcRelAggregates(create_guid(), owner_history, "Site Container", None, site, [building])
container_project = ifcfile.createIfcRelAggregates(create_guid(), owner_history, "Project Container", None, project, [site])

#Wall Input Conversations
def create_wall():

    text = []
    text_to_voice (f"Please enter X dimension\n")
    text = get_voice()
    x_dimension = float(re.findall('\d*\.?\d+', text)[0])
    text_to_voice (f" X dimension is {x_dimension}")

    text = []
    text_to_voice (f"Please enter y dimension\n")
    text = get_voice()
    y_dimension = float(re.findall('\d*\.?\d+', text)[0])
    #print(y_dimension)
    text_to_voice (f" Y dimension is {y_dimension}")

    text = []
    text_to_voice (f"Please enter the height\n")
    text = get_voice()
    z_dimension = float(re.findall('\d*\.?\d+', text)[0])
    #print(z_dimension)
    text_to_voice (f" Height is {z_dimension}")

    text = []
    text_to_voice (f"Please give me the x and y values of the closest point to origin\n")
    text = get_voice()
    origin_x = float(re.findall('\d*\.?\d+', text)[0])
    origin_y = float(re.findall('\d*\.?\d+', text)[1])
    text_to_voice (f"Closest point is {origin_x} comma {origin_x}")

    #coordinate creation
    point_list_extrusion_area = [(origin_x, origin_y, 0.0), (x_dimension + origin_x, origin_y, 0.0), (x_dimension + origin_x, y_dimension + origin_y, 0.0), (origin_x, y_dimension+ origin_y, 0.0), (origin_x, origin_y, 0.0)]

    #Create Wall
    wall_placement = create_ifclocalplacement(ifcfile, relative_to=storey_placement)
    polyline = create_ifcpolyline(ifcfile, [(0.0, 0.0, 0.0), (5.0, 0.0, 0.0)])
    axis_representation = ifcfile.createIfcShapeRepresentation(context, "Axis", "Curve2D", [polyline])

    extrusion_placement = create_ifcaxis2placement(ifcfile, (0.0, 0.0, 0.0), (0.0, 0.0, 1.0), (1.0, 0.0, 0.0))
    solid = create_ifcextrudedareasolid(ifcfile, point_list_extrusion_area, extrusion_placement, (0.0, 0.0, 1.0), z_dimension)
    body_representation = ifcfile.createIfcShapeRepresentation(context, "Body", "SweptSolid", [solid])

    product_shape = ifcfile.createIfcProductDefinitionShape(None, None, [axis_representation, body_representation])

    wall = ifcfile.createIfcWallStandardCase(create_guid(), owner_history, "Wall", "An awesome wall", None, wall_placement, product_shape, None)

    # Write the contents of the file to disk
    ifcfile.write(filename)

    return x_dimension, y_dimension, z_dimension, origin_x, origin_y
